In [93]:
import numpy as np 
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from scipy.io import arff
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.spatial.distance import pdist, cdist
from sklearn.metrics.pairwise import euclidean_distances, pairwise_distances
from sklearn.utils import check_random_state
from scipy.spatial import distance
from tslearn.soft_dtw_fast import *
from tslearn.cysax import *
from tslearn.metrics import dtw, dtw_path,dtw_path_from_metric,soft_dtw
from tslearn.utils import *


# Loading Training and Testing Data Source

In [94]:
data1 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension1_TRAIN.arff')
data2 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension2_TRAIN.arff')
data3 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension3_TRAIN.arff')

Test_data1 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension1_TEST.arff')
Test_data2 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension2_TEST.arff')
Test_data3 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension3_TEST.arff')

In [95]:
df_D1 = pd.DataFrame(data1[0])
df_D2 = pd.DataFrame(data2[0])
df_D3 = pd.DataFrame(data3[0])

Test_df_D1 = pd.DataFrame(Test_data1[0])
Test_df_D2 = pd.DataFrame(Test_data2[0])
Test_df_D3 = pd.DataFrame(Test_data3[0])

## Make Over the data, as desired

In [96]:
df_D1 =df_D1.apply(lambda x : x.fillna(method='ffill'), axis=1) 
df_D2 =df_D2.apply(lambda x : x.fillna(method='ffill'), axis=1)
df_D3 =df_D3.apply(lambda x : x.fillna(method='ffill'), axis=1) 

Test_df_D1 =Test_df_D1.apply(lambda x : x.fillna(method='ffill'), axis=1) 
Test_df_D2 =Test_df_D2.apply(lambda x : x.fillna(method='ffill'), axis=1)
Test_df_D3 =Test_df_D3.apply(lambda x : x.fillna(method='ffill'), axis=1) 

In [97]:
df_D1 = df_D1.drop('target', axis=1)
df_D2 = df_D2.drop('target', axis=1)
df_D3 = df_D3.drop('target', axis=1)

Test_df_D1 = Test_df_D1.drop('target', axis=1)
Test_df_D2 = Test_df_D2.drop('target', axis=1)
Test_df_D3 = Test_df_D3.drop('target', axis=1)

In [98]:
Sort_Dim_1 = np.expand_dims(df_D1.values.reshape(-1),axis=1)
Sort_Dim_2 = np.expand_dims(df_D2.values.reshape(-1),axis=1)
Sort_Dim_3 = np.expand_dims(df_D3.values.reshape(-1),axis=1)

Sort_Test_Dim_1 = np.expand_dims(Test_df_D1.values.reshape(-1),axis=1)
Sort_Test_Dim_2 = np.expand_dims(Test_df_D2.values.reshape(-1),axis=1)
Sort_Test_Dim_3 = np.expand_dims(Test_df_D3.values.reshape(-1),axis=1)

In [99]:
Training_View = np.concatenate((Sort_Dim_1, Sort_Dim_2),axis=1)
Training_View = np.concatenate((Training_View, Sort_Dim_3),axis=1)

In [100]:
Training_View.shape

(258804, 3)

In [101]:
Testing_View = np.concatenate((Sort_Test_Dim_1, Sort_Test_Dim_2),axis=1)
Testing_View = np.concatenate((Testing_View, Sort_Test_Dim_3),axis=1)

In [102]:
finalarray_test_data = Testing_View[:][0:182]

In [103]:
finalarray_test_data.shape

(182, 3)

## Initializing the data

In [104]:
Total_rows = 258804
Sample_Percentage = 0
Number_of_run= 0
Length_RTS = 0
Training_Results_List = []
Testing_Results_List = []
RTS_short_list_Results_List = []
Starting_point = Testing_View[:][0:1]
End_point = Testing_View[:][181:182]

## Random TS

In [105]:
def Calculate_Percentage(Sample_Percentage):
    Refining_Percentage= Total_rows * (Sample_Percentage/100)
    Refining_Percentage = round(Refining_Percentage)   
    return Refining_Percentage

In [106]:
def Random_Part(Refining_Percentage,Length_RTS):
    Sub_Sampling = Training_View[:][0:Refining_Percentage]
    number_of_rows = Sub_Sampling.shape[0]
    random_indices = np.random.choice(number_of_rows, size=Length_RTS, replace=False)
    random_rows = Sub_Sampling[random_indices, :]
    return random_rows
    

In [107]:
def Adding_start_end_point(random_rows):
    Output_TS = np.concatenate((Starting_point, random_rows),axis=0)
    Output_TS = np.concatenate((Output_TS, End_point),axis=0)
    return Output_TS

In [109]:
def Calculate_Random_DTW(Output_TS,Len_RTS):
    path, dist = dtw_path(finalarray_test_data, Output_TS)
    if(dist>0 and Len_RTS >100 ):
        
        Training_Results_List.append(dist)  
    else:
        RTS_short_list_Results_List.append(dist)

In [111]:
for x in range(1,6):
    Refining_Percentage = Calculate_Percentage(20*x)
    random_rows = Random_Part(Refining_Percentage,180)
    Output_TS = Adding_start_end_point(random_rows)
    Calculate_Random_DTW(Output_TS,180)


In [112]:
for x in range(1,6):
    Refining_Percentage = Calculate_Percentage(20*x)
    random_rows = Random_Part(Refining_Percentage,70)
    Output_TS = Adding_start_end_point(random_rows)
    Calculate_Random_DTW(Output_TS,70)



In [113]:
Training_Results_List

[22.64698926927984,
 23.269931345890047,
 24.504007404939326,
 22.132534240115273,
 21.838837589021086]

In [114]:
RTS_short_list_Results_List

[14.649184184519426,
 15.517276040180505,
 15.49228193729055,
 15.917089553049989,
 15.027249368013726]

# Pseudo TS

In [65]:
def generate_similarity_matrix(Complete_PTS):
    ED_Matrix = distance.cdist(Complete_PTS, Complete_PTS, 'euclidean')
    return ED_Matrix

In [66]:
def generate_graph(similarity_output):
    G = nx.from_numpy_matrix(similarity_output, create_using=nx.Graph)
    layout = nx.spring_layout(G)
    sizes = len(similarity_output)
    return G

In [67]:
def generate_MST(G):
    layout = nx.spring_layout(G)
    T=nx.minimum_spanning_tree(G)
    return T

In [68]:
def generate_Dijkstra(T):
    Dij = nx.dijkstra_path(T,0,181)
    return Dij

In [79]:
def doing_miscellaneous_work(Dijkstra_output):
    a = []
    for x in range(len(Dijkstra_output)):
    
        Index = Dijkstra_output[x]
        a.append(list(Complete_PTS[Index]))
    P_TS = np.array(a)
    return P_TS

In [80]:
def generate_DTW_PTS(Miscellaneous_output):
    path, dist = dtw_path(finalarray_test_data, Miscellaneous_output)
    if(dist>0):
        Testing_Results_List.append(dist)     

In [81]:
def Adding_start_end_point_PTS(Pseudo_TS):
    Output_PTS = np.concatenate((Starting_point, Pseudo_TS),axis=0)
    Output_PTS = np.concatenate((Output_PTS, End_point),axis=0)
    return Output_PTS

In [82]:
def Run_all_fun(Complete_PTS):
    similarity_output = generate_similarity_matrix(Complete_PTS)
    graph_output = generate_graph(similarity_output)
    MST_output_1 =  generate_MST(graph_output)
    Dijkstra_output = generate_Dijkstra(MST_output_1)
    Miscellaneous_output = doing_miscellaneous_work(Dijkstra_output)
    generate_DTW_PTS(Miscellaneous_output)

In [91]:
for x in range(1,6):
    i = 181
    if(x==1):
        
        Sub_Sampling_PTS = Training_View[:][1:181*x]
        Pseudo_TS = Sub_Sampling_PTS
        Complete_PTS = Adding_start_end_point_PTS(Pseudo_TS)
        Run_all_fun(Complete_PTS)  
    else:
        
        Sub_Sampling_PTS = Training_View[:][1+i:181*x]
        Pseudo_TS = Sub_Sampling_PTS
        Complete_PTS = Adding_start_end_point_PTS(Pseudo_TS)
        Run_all_fun(Complete_PTS) 
 

In [92]:
Testing_Results_List  

[10.05884167027412,
 9.3343652050839,
 9.78243724869677,
 9.797855373367529,
 9.80363631840538]